In [4]:
pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.3.0
    Uninstalling typeguard-4.3.0:
      Successfully uninstalled typeguard-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LayerNormalization, Dropout, MultiHeadAttention
import librosa
import numpy as np

# --- Step 1: Preprocessing Function ---

def preprocess_audio(file_path, sr=16000, n_mels=128, hop_length=512):
    """Convert raw audio into a Mel-Spectrogram."""
    audio, sr = librosa.load(file_path, sr=sr)
    mel_spec = librosa.feature.melspectrogram(audio, sr=sr, n_mels=n_mels, hop_length=hop_length)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db.T  # Transpose for time-major format


# --- Step 2: Positional Encoding ---

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(sequence_length, d_model)

    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        return pos * angle_rates

    def positional_encoding(self, sequence_length, d_model):
        angle_rads = self.get_angles(np.arange(sequence_length)[:, np.newaxis],
                                     np.arange(d_model)[np.newaxis, :],
                                     d_model)

        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])  # sin on even indices
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])  # cos on odd indices

        pos_encoding = angle_rads[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]


# --- Step 3: Multihead Attention Anonymization Model ---

def build_transformer_model(input_shape, d_model, num_heads, ff_dim, num_layers, dropout_rate):
    inputs = tf.keras.Input(shape=input_shape)

    # Positional Encoding
    x = PositionalEncoding(input_shape[0], d_model)(inputs)

    # Multi-Head Attention + Feed Forward Layers
    for _ in range(num_layers):
        # Layer Normalization and Multi-Head Attention
        x1 = LayerNormalization(epsilon=1e-6)(x)
        attention_out = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x1, x1)
        x2 = Dropout(dropout_rate)(attention_out)
        x = tf.keras.layers.Add()([x, x2])  # Skip Connection

        # Feed Forward Network
        x1 = LayerNormalization(epsilon=1e-6)(x)
        ff_out = Dense(ff_dim, activation='relu')(x1)
        ff_out = Dense(d_model)(ff_out)
        x2 = Dropout(dropout_rate)(ff_out)
        x = tf.keras.layers.Add()([x, x2])  # Skip Connection

    # Output layer (same shape as input)
    outputs = Dense(input_shape[-1])(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


# --- Step 4: Define Loss and Custom Training Step ---

def loss_fn(y_true, y_pred, alpha=0.5):
    """Loss function combining content preservation and anonymization."""
    content_loss = tf.reduce_mean(tf.square(y_true - y_pred))  # Content preservation
    anonymization_loss = -tf.reduce_mean(tf.reduce_sum(tf.abs(y_true - y_pred), axis=-1))  # Encourage dissimilarity
    total_loss = alpha * content_loss + (1 - alpha) * anonymization_loss
    return total_loss


# --- Step 5: Data and Model Training ---

# Example usage with dummy data
sample_rate = 16000
mel_spectrogram_shape = (128, 128)  # (time, mel-frequency bins)
d_model = 128  # Transformer dimensionality
num_heads = 4  # Number of attention heads
ff_dim = 512  # Feed forward layer size
num_layers = 4  # Number of transformer layers
dropout_rate = 0.1

# Build the model
transformer_model = build_transformer_model(mel_spectrogram_shape, d_model, num_heads, ff_dim, num_layers, dropout_rate)

# Compile the model
transformer_model.compile(optimizer='adam', loss=loss_fn)

# Dummy data for illustration (use real Mel-spectrograms for real data)
batch_size = 16
dummy_data = np.random.random((batch_size, mel_spectrogram_shape[0], mel_spectrogram_shape[1]))
dummy_labels = np.random.random((batch_size, mel_spectrogram_shape[0], mel_spectrogram_shape[1]))

# Train the model (replace with
